In [1]:
import requests
from datetime import datetime

import os
from apiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.oauth2 import service_account
from googleapiclient.discovery import build

def setup_gdrive(credentials_secret):
    with open('credentials.json', 'w') as f:
        f.write(os.environ[credentials_secret])
        
    # Load the credentials from the JSON key file
    credentials = service_account.Credentials.from_service_account_file(
        'credentials.json',
        scopes=['https://www.googleapis.com/auth/drive']
    )

    drive_service = build('drive', 'v3', credentials=credentials)
    return drive_service



In [2]:
with open('/Users/przivic/Downloads/quetiene-3acd3a3a2117.json') as f:
    credentials = f.read()

os.environ['GDRIVE_SECRET'] = credentials

In [3]:
with open('/Users/przivic/Downloads/enre-414003-3724f64a117f.json') as f:
    credentials = f.read()

os.environ['NEW_PROD_GDRIVE_SECRET'] = credentials

In [4]:
prod_drive_service = setup_gdrive('GDRIVE_SECRET')
new_prod_drive_service = setup_gdrive('NEW_PROD_GDRIVE_SECRET')

In [10]:
a = prod_drive_service.files()
a.list?

In [20]:
a = prod_drive_service.files()

In [23]:
all_files = []
page_token = None
while True:
    print('a')
    response = prod_drive_service.files().list(pageToken=page_token).execute()
    all_files.extend(response['files'])
    if 'nextPageToken' not in response: break
    page_token = response['nextPageToken']

a
a


In [27]:
import re

def parse_fname_date(fname):
    date_pat = re.compile('(\d{4}-\d{2}-\d{2}T\d{2})')
    return datetime.strptime(date_pat.search(fname).group(0), '%Y-%m-%dT%H')


from collections import Counter

cnt = Counter()
for file in all_files:
    cnt[parse_fname_date(file['name'])] += 1

In [32]:
import os

if not os.path.exists('data'): os.makedirs('data')

for file in all_files:
    file_id = file['id']
    request = prod_drive_service.files().get_media(fileId=file_id)
    if os.path.exists('data/' + file['name']): 
        print('skipping already downloaded file',  file['name'])
        continue
        
    with open('data/' + file['name'], 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}%. {file_id}")

Download 100%. 1rU_Ww15J-1X44F3dl4B0SXNqIM1NO2cs
Download 100%. 1UjdL2oQJ20Tt5bKuiJzvFbN1XL7kAfEr
Download 100%. 1Kpy0aZQ7DZzZWoApL3MwC9OpCcswlWzD
Download 100%. 1Y7m_NQdbhF2A_EnTHpM3VTELwODwUymU
Download 100%. 1oPUa6wsV2hteX0Q93jCGtudvd47XHy3m
Download 100%. 1lXsN8Z4kCSu-piUyaLY8X9w_QC738l5J
Download 100%. 1LuZa5X-QLq7nlKLCzCfp2qFFlSSscB3W
Download 100%. 1aETljUHlgyGa_wClNq50xPME4c9JB6PC
Download 100%. 1f6zoBozwrAMaHvwiU1hKH4gWFxko20ai
Download 100%. 1nORJMayorKQ9eImklmHfuI0d8MwLW8aQ
Download 100%. 1sOObJdSwvHVzbPuY33MbEpHarIC9hcGH
Download 100%. 1toFkwowvYoYhW6J098ywRYiZiN86c1VJ
Download 100%. 1XcZyoumH0WZP-VaODfhJcUK3yvRAx8uO
Download 100%. 16l4DN02Qx6AUIpa2y6iy95XyTzA7vdBy
Download 100%. 1YS6DuwdQgtW6liknpyGWrI3NdIvSA281
Download 100%. 1LjGzzpHXgvXjkNzrv4fjggaJsjOb_RD2
Download 100%. 1NM9fucAKFdYvQPspmy92FIdMeN3B5YQ3
Download 100%. 1khDqkdNVJKP8TJGtHUYF4JfYvzJ156rl
Download 100%. 15wIm33tDCjiqXEkNzuSZJWgQJyCuA43U
Download 100%. 1Y7wo4_vNQQN-TWswKH27eS4-juRFeNiw
Download 100%. 1AroS

In [33]:
!ls data | wc -l

     140


In [35]:
len(all_files)

148

In [36]:
def upload_fname(drive_service, fname):
    file_metadata = {'name': fname}
    media = MediaFileUpload(fname, mimetype='text/plain')
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f"File ID: {file.get('id')}")

In [37]:
from tqdm import tqdm

for fname in tqdm(os.listdir('data')):
    upload_fname(new_prod_drive_service, 'data/' + fname)

  1%|▋                                                                                                 | 1/140 [00:01<03:50,  1.66s/it]

File ID: 1_feIWaD20rQYkJ5_g2pIZ_SdTtd4ryVO


  1%|█▍                                                                                                | 2/140 [00:03<03:30,  1.53s/it]

File ID: 1s0IPaAN_eRV63n1KqGMnd-ToIL1XvrGx


  2%|██                                                                                                | 3/140 [00:04<03:36,  1.58s/it]

...

File ID: 18Q_XzLgPLMQGq0Nx21OZj3Zo-OcVoUcM


 99%|███████████████████████████████████████████████████████████████████████████████████████████████▎| 139/140 [03:30<00:01,  1.53s/it]

File ID: 1rv9-SHyQS77709RAHiWH3btjEnmDBC1-


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [03:31<00:00,  1.51s/it]

File ID: 1chR1zOp5rz4H1oVSNvrlORiB1A2gL9Kx


In [38]:
response = new_prod_drive_service.files().list().execute()
response

{'nextPageToken': '~!!~AI9FV7Q2h0p3PLM6TuwymS7Vlkos7MjxM_YEXrVrldKU4ltFRB3Gao9ufh120h7XOubHtbm7vUsVW1Ivx12knUV_ZHKLQVu0uLFtPgJy8ZYzuLWUE42r8dOJasXg8YxogRdnwsSrXUcSHnzBBw1Nc_Bx8H50XTqjd_trl1lf4SlZykHAGXLbMa0bTFmWMEVwwYxaqbB3WoKjX8AFfo0JO3RLYiJ-oyjPKbQtbmd1cOpuBodu00SXarIBjqfKmDjLnIlKN6n9bkc4_KruSWv8U54vO6pGfj-vB5uh1vvi_yM28fxwg8N1XDIRM0G6iPwnLXOtkL6zxduJ0DIfuTvsgqTHVPeCgizNMbztN4nP9B3owY6Jdztq51aqlPRBPh5eTbwd5EoOb-z8',
 'kind': 'drive#fileList',
 'incompleteSearch': False,
 'files': [{'kind': 'drive#file',
   'mimeType': 'text/plain',
   'id': '1chR1zOp5rz4H1oVSNvrlORiB1A2gL9Kx',
   'name': 'data/demanda-enre-2024-02-12T02-1077.txt.gz'},
  {'kind': 'drive#file',
   'mimeType': 'text/plain',
   'id': '1rv9-SHyQS77709RAHiWH3btjEnmDBC1-',
   'name': 'data/demanda-enre-2024-02-11T22-1078.txt.gz'},
  {'kind': 'drive#file',
   'mimeType': 'text/plain',
   'id': '18Q_XzLgPLMQGq0Nx21OZj3Zo-OcVoUcM',
   'name': 'data/demanda-enre-2024-02-11T18-1077.txt.gz'},
  {'kind': 'drive#file',
   'mimeType

In [41]:
all_files = []
page_token = None
while True:
    print('a')
    response = new_prod_drive_service.files().list(pageToken=page_token).execute()
    all_files.extend(response['files'])
    if 'nextPageToken' not in response: break
    page_token = response['nextPageToken']

a
a


In [42]:
len(all_files)

144